<a href="https://colab.research.google.com/github/shekharkoirala/machinelearning_algorithms_analysis/blob/master/stanfordcourse/tf_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

Get MNIST data

In [13]:
!curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9680k  100 9680k    0     0  29.3M      0 --:--:-- --:--:-- --:--:-- 29.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28881  100 28881    0     0   376k      0 --:--:-- --:--:-- --:--:--  376k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1610k  100 1610k    0     0  7352k      0 --:--:-- --:--:-- --:--:-- 7352k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4542  100  4542    0     0  94625      0 --:--:-- --:--:-- --:--:-- 96638


In [0]:
!gunzip t*-ubyte.gz

In [15]:
!ls

data	     t10k-images-idx3-ubyte  train-images-idx3-ubyte
sample_data  t10k-labels-idx1-ubyte  train-labels-idx1-ubyte


In [16]:

!sudo mkdir data
!sudo mkdir data/mnist

mkdir: cannot create directory ‘data’: File exists


In [0]:
!sudo cp t10k-images-idx3-ubyte ./data/mnist/
!sudo cp t10k-labels-idx1-ubyte ./data/mnist/
!sudo cp train-images-idx3-ubyte ./data/mnist/
!sudo cp train-labels-idx1-ubyte ./data/mnist/

In [18]:
!ls data/mnist

t10k-images-idx3-ubyte	train-images-idx3-ubyte
t10k-labels-idx1-ubyte	train-labels-idx1-ubyte


In [0]:
import struct 

def parse_data(path, dataset, flatten):
  label_path = os.path.join(path , dataset + '-labels-idx1-ubyte' )
  with open(label_path, "rb") as file:
    _ , num = struct.unpack(">II", file.read(8))
    labels = np.fromfile(file, dtype= np.int8)
    new_labels = np.zeros((num, 10))
    new_labels[np.arange(num), labels] = 1
  
  img_path = os.path.join(path, dataset +'-images-idx3-ubyte')
  with open(img_path, "rb") as file:
    _, num, rows, cols = struct.unpack(">IIII", file.read(16))
    image = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols)
    image = image.astype(np.float32) / 255.0
    if flatten:
      image = image.reshape([num, -1])
  return image,  new_labels

def get_mnist_data(batchsize, num_train=55000, flatten=True):
  data_folder= "data/mnist"
  imgs, labels = parse_data(data_folder, "train", flatten)
  indices = np.random.permutation(labels.shape[0])
  train_idx, val_idx = indices[:num_train], indices[num_train:]
  train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
  val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
  test = parse_data(data_folder, 't10k', flatten)
  train , val , test = (train_img, train_labels), (val_img, val_labels), test
  
  train_data = tf.data.Dataset.from_tensor_slices(train)
  train_data = train_data.shuffle(1000)
  train_data = train_data.batch(batchsize)
  
  test_data = tf.data.Dataset.from_tensor_slices(train)
  test_data = test_data.batch(batchsize)
  return train_data, test_data
